In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from myfile import remove_outlier,isnull_values_sum,plot_box_plot,read_csv_morceaux,group_and_loc_group

In [2]:
pd.set_option("display.precision", 2)          
pd.set_option("display.max.columns", None)    
pd.set_option("display.max.rows", None)
df =pd.read_csv('../../P3/DATA/Projet_habitudes_alimentaires.csv', sep='\t', encoding='utf-8')

In [3]:
# Groupee par nutriscore
groupby = ['salt_100g']#nutriscore_grade']
[grouped, df_grouped] = group_and_loc_group(df,groupby)
 

KeyError: 'salt_100g'

In [4]:
df.head()

,Unnamed: 0,product_name,countries_en
0,0,jeunes pousses,France
1,2,Cacao,France
2,6,Sfiudwx,France
3,7,Sauce Sweety chili 0%,France
4,8,Mini coco,France
